In [38]:
import numpy as np
from pydrake.all import (
    DiagramBuilder, AddMultibodyPlantSceneGraph, Parser,
    Meshcat, MeshcatVisualizer, MeshcatVisualizerParams,
    Simulator, KinematicTrajectoryOptimization, Solve, SolverOptions, SnoptSolver
)

from manipulation import ConfigureParser, running_as_notebook

In [39]:
meshcat = Meshcat()
print("Meshcat URL:", meshcat.web_url())

INFO:drake:Meshcat listening for connections at http://localhost:7007


Meshcat URL: http://localhost:7007


In [40]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant)
ConfigureParser(parser)
parser.AddModelsFromUrl("package://drake_models/pr2_description/urdf/pr2_simplified.urdf")
plant.Finalize()

MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph.get_query_output_port(),
    meshcat,
    MeshcatVisualizerParams(delete_on_initialization_event=False),
)

diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
plant_context = plant.GetMyMutableContextFromRoot(context)

simulator.Initialize()
simulator.AdvanceTo(0.01)
nq = plant.num_positions()

In [57]:
q_hi

array([       inf,        inf,        inf, 0.31      , 3.007     ,
       1.39626   , 0.71460184, 1.3963    , 0.8       , 0.        ,
              inf, 0.        ,        inf, 0.548     , 0.548     ,
       0.548     , 0.548     , 2.28539816, 1.3963    , 3.9       ,
       0.        ,        inf, 0.        ,        inf, 0.548     ,
       0.548     , 0.548     , 0.548     ])

In [58]:
q_now = plant.GetPositions(plant_context).copy()
q_lo = plant.GetPositionLowerLimits()
q_hi = plant.GetPositionUpperLimits()

q_start = q_now.copy()
q_goal = q_start.copy()
q_goal[0] = 2
q_goal[1] = 2
q_goal[2] = np.deg2rad(30.0)


In [59]:
velocity_lower_constraints = plant.GetVelocityLowerLimits()[:nq]
velocity_upper_constraints = plant.GetVelocityUpperLimits()[:nq]
velocity_lower_constraints[:3] = -2.0
velocity_upper_constraints[:3] = 2.0

In [60]:
N = 10
min_t = 0.5
max_t = 5.0
eps = 1e-6

kto = KinematicTrajectoryOptimization(nq, N)
prog = kto.get_mutable_prog()

kto.AddDurationCost(10.0)
kto.AddDurationConstraint(min_t, max_t)

kto.AddPositionBounds(
	plant.GetPositionLowerLimits()[:nq], 
	plant.GetPositionUpperLimits()[:nq]
)

kto.AddVelocityBounds(
	velocity_lower_constraints, 
	velocity_upper_constraints
)
kto.AddPathPositionConstraint(q_start-eps, q_start+eps, 0)
kto.AddPathPositionConstraint(q_goal-eps, q_goal+eps, 1)

solver = SnoptSolver()
opts = SolverOptions()
opts.SetOption(solver.id(), "minor feasibility tolerance", 8e-2)
result = solver.Solve(prog, solver_options=opts)

if not result.is_success():
	print("traj opt failed: no collision checking!")
	print("infeasible constraints", result.GetInfeasibleConstraintNames(prog))
	print("traj_opt solver name", result.get_solver_id().name())

traj = kto.ReconstructTrajectory(result)




In [ ]:
import time

if traj is None:
    print("No trajectory to animate.")
else:
    T = traj.end_time()
    ts = np.linspace(0.0, T, 120)
    for t in ts:
        q = traj.value(t).ravel()        # full q (length = nq)
        plant.SetPositions(plant_context, q)
        diagram.ForcedPublish(context)
        time.sleep(max(T/len(ts)/1.5, 0.01))  # gentle playback
    print("Done.")

Done.
